In [13]:
import MarketDataFetcher
import asyncio
import nest_asyncio
import utils
import json
import numpy as np
import pickle

nest_asyncio.apply()

obj = MarketDataFetcher.FuturesMarket()
data = asyncio.run(obj.fetch_klines_limit(symbol='BTCUSDT', interval='3m', limit=1_000))
array = np.array(data, float)

In [14]:
save_path = '/Users/cjupit/Desktop/btc.pickle'
with open(save_path, 'wb')as file:
    pickle.dump(array, file)

In [15]:
utils._info_kline_columns()

['0: Open Time',
 '1: Open',
 '2: High',
 '3: Low',
 '4: Close',
 '5: Volume',
 '6: Close Time',
 '7: Quote Asset Volume',
 '8: Number of Trades',
 '9: Taker Buy Base Asset Volume',
 '10: Taker Buy Quote Asset Volume',
 '11: Ignore']